This notebooks demonstrates basic usage of the fv3config package by doing two sequential 1-hour runs. It assumes that fv3config has been installed and that a fv3gfs-python docker image is locally available.

In [1]:
import fv3config
import yaml
import os
import shutil
import copy

In [2]:
model_image = 'us.gcr.io/vcm-ml/fv3gfs-python:latest'
config_filename = 'c12_config.yml'

#### Configuration dictionaries can be saved to and loaded from yaml

In [3]:
with open(config_filename, 'r') as f:
    initial_config = yaml.safe_load(f)
list(initial_config.keys())

['data_table',
 'diag_table',
 'experiment_name',
 'forcing',
 'orographic_forcing',
 'initial_conditions',
 'namelist']

In [4]:
initial_config['namelist']['coupler_nml']

{'atmos_nthreads': 1,
 'calendar': 'julian',
 'current_date': [2016, 8, 1, 0, 0, 0],
 'days': 0,
 'dt_atmos': 900,
 'dt_ocean': 900,
 'hours': 0,
 'memuse_verbose': True,
 'minutes': 30,
 'months': 0,
 'ncores_per_node': 32,
 'seconds': 0,
 'use_hyper_thread': True}

In [5]:
initial_config['diag_table']

'default'

Built-in options for diag_table are 'default', 'grid_spec' and 'no_output'. You can specify your own custom diag_table by using an absolute path that points to a file on your system:

In [6]:
initial_config['diag_table'] = os.path.abspath('diag_table_for_example_nb')

#### Change length of run to one hour

In [7]:
initial_config['namelist']['coupler_nml']['hours'] = 1
initial_config['namelist']['coupler_nml']['minutes'] = 0

#### Run model

In [8]:
initial_rundir = './initial_rundir/'
if os.path.isdir(initial_rundir):
    shutil.rmtree(initial_rundir)
os.makedirs(initial_rundir)
fv3config.run_docker(initial_config, initial_rundir, docker_image=model_image)

In [9]:
print(sorted(os.listdir(initial_rundir)))
print(sorted(os.listdir(os.path.join(initial_rundir, 'RESTART'))))

['INPUT', 'RESTART', 'aerosol.dat', 'atmos_dt_atmos.tile1.nc', 'atmos_dt_atmos.tile2.nc', 'atmos_dt_atmos.tile3.nc', 'atmos_dt_atmos.tile4.nc', 'atmos_dt_atmos.tile5.nc', 'atmos_dt_atmos.tile6.nc', 'co2historicaldata_2010.txt', 'co2historicaldata_2011.txt', 'co2historicaldata_2012.txt', 'co2historicaldata_2013.txt', 'co2historicaldata_2014.txt', 'co2historicaldata_2015.txt', 'co2historicaldata_2016.txt', 'data_table', 'diag_table', 'field_table', 'fv3config.yml', 'global_o3prdlos.f77', 'grb', 'input.nml', 'logfile.000000.out', 'sfc_emissivity_idx.txt', 'solarconstant_noaa_an.txt', 'stderr.log', 'stdout.log', 'time_stamp.out']
['coupler.res', 'fv_core.res.nc', 'fv_core.res.tile1.nc', 'fv_core.res.tile2.nc', 'fv_core.res.tile3.nc', 'fv_core.res.tile4.nc', 'fv_core.res.tile5.nc', 'fv_core.res.tile6.nc', 'fv_srf_wnd.res.tile1.nc', 'fv_srf_wnd.res.tile2.nc', 'fv_srf_wnd.res.tile3.nc', 'fv_srf_wnd.res.tile4.nc', 'fv_srf_wnd.res.tile5.nc', 'fv_srf_wnd.res.tile6.nc', 'fv_tracer.res.tile1.nc', 

#### Configure a run to initialize from end of previous run

In [ ]:
ic_directory = os.path.abspath(os.path.join(initial_rundir, 'RESTART'))
restart_config = copy.deepcopy(initial_config)
restart_config['initial_conditions'] = ic_directory
restart_config['namelist']['fv_core_nml']['warm_start'] = True
restart_config['namelist']['fv_core_nml']['external_ic'] = False
restart_config['namelist']['fv_core_nml']['nggps_ic'] = False
restart_config['namelist']['fv_core_nml']['make_nh'] = False
restart_config['namelist']['fv_core_nml']['mountain'] = True
restart_config['namelist']['fv_core_nml']['na_init'] = 0

restart_rundir = './restart_rundir'
if os.path.isdir(restart_rundir):
    shutil.rmtree(restart_rundir)
os.makedirs(restart_rundir)
fv3config.run_docker(restart_config, restart_rundir, docker_image=model_image)